In [ ]:
### TO BE MADE INTO A SEPARATE PYTHON FILE ####

import streamlit as st

def search(input_keyword):
    return[1,2]

def main():
    st.title("Search Myntra Fashion Products")
    search_query = st.text_input("Enter your search query")


    if st.button("Search"):
        if search_query:
            results = search(search_query)

            st.subheader("Search Results")
            for result in results:
                with st.container():
                    if '_source' in result:
                        try:
                            st.header(f"{result[0]}")
                        except Exception as e:
                            print(e)
                        
                        try:
                            st.write(f"Description: {result[2]}")
                        except Exception as e:
                            print(e)
                        st.divider()
 
if __name__ == "__main__":
    main()

In [1]:
!pip install pymongo

In [2]:
!pip install python-dotenv

In [3]:
!pip install mysql-connector-python

In [5]:
import json
import os
import mysql.connector
import pymongo
import mysql.connector

from pymongo import MongoClient
from bson import ObjectId
from dotenv import load_dotenv
from datetime import datetime
from pprint import pprint
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from datetime import datetime

In [6]:
load_dotenv()

True

## MONGODB

In [7]:
mongo_username = os.environ.get("MONGOUSERNAME")
mongo_password = os.environ.get("MONGOPASSWORD")

uri = "mongodb+srv://{}:{}@twitter.qlewowk.mongodb.net/?retryWrites=true&w=majority&appName=twitter".format(mongo_username, mongo_password)

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Connected to MongoDB!")
except Exception as e:    print(e)

Connected to MongoDB!


In [10]:
db = client["sample"]
collection = db["tweets"]

In [11]:
file_name = "corona-out-2"

In [ ]:
lines=0
num_tweets=0
num_retweets=0
print_tweet = 0
print_retweet = 0
tweets = {}

with open(file_name, "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            lines = lines + 1
            if ( data['text'].startswith('RT') ):
                num_retweets += 1
                if (print_retweet < 2):
                    print_retweet += 1
                    print('RETWEET\n','id--',data['id'],'text--',data['text'],'\n')
                    print('User-- ',data['user'],'\n')
                    print('retweetstatus--',data['retweeted_status'],'\n')
            else:
                num_tweets += 1
                if (print_tweet < 2):
                    print_tweet += 1
                    print('TWEET\n','id--',data['id'],'text--',data['text'],'\n')
                    print('User-- ',data['user'],'\n')
                    print('retweetstatus--',data['retweeted_status'],'\n')
            if (data['id_str'] not in tweets):
                tweets[data['id_str']] = data
        except:
            continue

print('num of lines=',lines, 'num of tweets=',num_tweets, 'num of retweets=',num_retweets)
print('num of unique tweets/retweets=',len(tweets.keys()))

In [28]:
pprint(data["retweeted_status"])

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Apr 12 17:22:04 +0000 2020',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'dlvr.it/RTf5SD',
                        'expanded_url': 'http://dlvr.it/RTf5SD',
                        'indices': [55, 78],
                        'url': 'https://t.co/eDbMsxFA84'}],
              'user_mentions': []},
 'favorite_count': 108,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1249387319229988865,
 'id_str': '1249387319229988865',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'in',
 'place': None,
 'possibly_sensitive': False,
 'quote_count': 7,
 'reply_count': 12,
 'retweet_count': 86,
 'retweeted': False,
 'source': '<a href="https://dlvrit.com/" rel="nofollow">dlvr.it</a>',
 'text': 'Lagi, 2 Dokter Senior 

In [33]:
tweet_fields = ["id_str", "created_at", "favorited", "favorite_count", "lang", "retweet_count",
                "quote_count", "reply_count", "retweet_count", "entities", "text"]

def get_tweets(file_name, retweets = False):
    lines=0
    num_tweets=0
    num_retweets=0
    print_tweet = 0
    print_retweet = 0
    tweets = []
    exceptions = []
    unique_ones = set()
    with open(file_name, "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)
                final_data = {}
                if data["id"] not in unique_ones:
                    unique_ones.add(data["id"])
                    lines = lines + 1
                    final_data["_id"] = data["id"]
                    final_data["user_id"] = data["user"]["id"]
                    for field in tweet_fields:
                        final_data[field] = data[field]
                    if (data['text'].startswith('RT')):
                        num_retweets += 1
                        if retweets and num_retweets == 1:
                                pprint(data["retweeted_status"])
                        retweeted_status = {}
                        retweeted_status["_id"] = data["retweeted_status"]["id"]
                        retweeted_status["user_id"] = data["retweeted_status"]["user"]["id"]
                        for field in tweet_fields:
                            retweeted_status[field] = data["retweeted_status"][field]
                        final_data["retweeted_status"] = retweeted_status
                        final_data["retweeted"] = True
                    else:
                        num_tweets += 1
                    if final_data:
                        tweets.append(final_data)
            except Exception as e:
                exceptions.append(e)

    print('num of lines=',lines, 'num of tweets=',num_tweets, 'num of retweets=',num_retweets)
    print(len(tweets),  " tweets captured")
    print(len(exceptions), " exceptions")
    return tweets

t = get_tweets(file_name, retweets = True)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Apr 12 16:48:01 +0000 2020',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'favorite_count': 50,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1249378751349231616,
 'id_str': '1249378751349231616',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': True,
 'lang': 'en',
 'place': None,
 'quote_count': 1,
 'quoted_status': {'contributors': None,
                   'coordinates': None,
                   'created_at': 'Sun Apr 12 12:36:30 +0000 2020',
                   'entities': {'hashtags': [],
                                'symbols': [],
                                'urls': [],
                                'user_mentions': []},
                   'favorite_count': 33006,
                   'favorited': False,
                 

In [30]:
def populate_tweets(tweets, client, db, collection):
    result = collection.insert_many(tweets)
    print("Tweets inserted into MongoDB collection")

In [15]:
database_names = client.list_database_names()

for database_name in database_names:
    print(database_name)

sample
admin
local


In [16]:
collection_names = db.list_collection_names()

print("Collections in the database:")
for collection_name in collection_names:
    print(collection_name)


Collections in the database:
tweets


In [17]:
document_count = collection.count_documents({})
print("Number of documents in the collection:", document_count)

Number of documents in the collection: 7349


In [34]:
tweets = get_tweets(file_name)

num of lines= 18506 num of tweets= 7349 num of retweets= 11157
18503  tweets captured
18521  exceptions


In [35]:
tweets[1]

{'_id': 1249403768023678982,
 'user_id': 1225145123920588805,
 'id_str': '1249403768023678982',
 'created_at': 'Sun Apr 12 18:27:25 +0000 2020',
 'favorited': False,
 'favorite_count': 0,
 'lang': 'en',
 'retweet_count': 0,
 'quote_count': 0,
 'reply_count': 0,
 'entities': {'hashtags': [],
  'urls': [],
  'user_mentions': [{'screen_name': 'lale_karanfil',
    'name': 'Karanfil Lale',
    'id': 1087735689091928064,
    'id_str': '1087735689091928064',
    'indices': [3, 17]}],
  'symbols': []},
 'text': 'RT @lale_karanfil: In Turkey, there are 300 thousand prisoners and 150 thousand prison employees in prisons. \nPrisons are the most risky pl…',
 'retweeted_status': {'_id': 1249397541596286979,
  'user_id': 1087735689091928064,
  'id_str': '1249397541596286979',
  'created_at': 'Sun Apr 12 18:02:41 +0000 2020',
  'favorited': False,
  'favorite_count': 6,
  'lang': 'en',
  'retweet_count': 14,
  'quote_count': 0,
  'reply_count': 1,
  'entities': {'hashtags': [],
   'urls': [{'url': 'h

In [37]:
populate_tweets(tweets, client, db, collection)

Tweets inserted into MongoDB collection


In [38]:
keyword = "cat"
query = {"text": {"$regex": keyword, "$options": "i"}}

matching_tweets = collection.find(query)

count = 0
for tweet in matching_tweets:
    count += 1
    pprint(tweet['text'])

print(" ")
print("{} tweets found".format(count))

('RT @KancutMeleduk: Corona selesai sepertinya tempat ini asik buat staycation '
 '😲😲😲. \n'
 '\n'
 'Location Villa Hanoman Batu - Malang https://t.co/PARRy…')
("@marcopastore Logica da mentecatto e mentecatto chi l'apprezza. Il vostro "
 'problema è la psicosi, non il corona. Curatevi')
('RT @3arbiiyaa_M3: On est dans black mirror là ? Jamais de la vie je '
 "telecharge leur applications d'espions")
('RT @victimeducators: As the corona spreads so fast, the incarcerated '
 'lawyers, judges, prosecutors, teachers, officers must be released imme…')
('@ericat_lol É pq com o corona, as org tao cortando gastos ai ao invés de '
 'Taric, eles pickam o Tapobre')
('RT @lilduval: CDC update: you can possibly catch the corona virus by looking '
 'someone directly in the eyes for more than 3 seconds')
('RT @KancutMeleduk: Corona selesai sepertinya tempat ini asik buat staycation '
 '😲😲😲. \n'
 '\n'
 'Location Villa Hanoman Batu - Malang https://t.co/PARRy…')
('RT @Artie_B: @realDonaldTrump @RealJame

## SQL

In [ ]:
sql_username = os.environ.get("SQLUSERNAME")
sql_password = os.environ.get("SQLPASSWORD")

conn = mysql.connector.connect(
    host = "localhost",
    user = sql_username,
    password = sql_password
)

cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS twitter_user_data")
cursor.execute("USE twitter_user_data")

In [ ]:
table_name = "users_info"
drop_table_query = f"DROP TABLE IF EXISTS {table_name};"

try:
    cursor.execute(drop_table_query)
    print(f"Table '{table_name}' dropped successfully.")

    conn.commit()

except mysql.connector.Error as error:
    print(f"Failed to drop table: {error}")

In [ ]:
create_table_query = """
CREATE TABLE IF NOT EXISTS users_info (
    id BIxGINT PRIMARY KEY,
    name VARCHAR(255),
    screen_name VARCHAR(255),
    location VARCHAR(255),
    description TEXT,
    verified BOOLEAN,
    followers_count INT,
    friends_count INT,
    created_at DATETIME
)
"""

cursor.execute(create_table_query)

In [ ]:
def insert_user_data(user, cursor):

    # Insert the user data into the table
    insert_query = """
    INSERT IGNORE INTO users_info (id, name, screen_name, location, description, verified, followers_count, friends_count, created_at)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    verified = 1 if user["verified"] else 0
    datetime_object = datetime.strptime(user["created_at"], '%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S')

    cursor.execute(insert_query, (
        user["id"],
        user["name"],
        user["screen_name"],
        user["location"],
        user["description"],
        verified,
        user["followers_count"],
        user["friends_count"],
        datetime_object
    ))

insert_user_data(data['user'], cursor)    

In [ ]:
select_query = "SELECT * FROM users_info"

cursor.execute(select_query)

rows = cursor.fetchall()

for row in rows:
    print(row)

In [ ]:
cursor.close()
conn.close()